In [12]:
!pip install mysql-connector-python

In [16]:
import mysql.connector

In [ ]:
import getpass
db_user = getpass.getpass('Enter username')
db_password = getpass.getpass('Enter password')

Enter username········


In [ ]:
config = {
   'user': db_user,
   'password': db_password,
   'host': 'localhost',
   'port': 3306,
   'database': 'university',
   'raise_on_warnings': True                    
}

In [ ]:
db_connection = mysql.connector.connect(**config)

In [ ]:
cur = db_connection.cursor()

# Activity 1

Suppose you are a staff member wanting to create a new ballot. In this database system, there are a couple elements to consider. First, the staff must create a ballot with a specified unique 4 character name and time frame as such below.

In [ ]:
query = "INSERT INTO ballots(ballot_name, date_start, date_end) VALUES ('NUM1', '2022-11-01 00:00:00', '2022-12-01 11:59:00');"
cur.execute(query)

Next the staff will need to add questions to the ballot. A ballot can have multiple questions. A ballot question must contain
the ballot name, questions number, and question text.

In [ ]:
query = "INSERT INTO ballot_questions(ballot_name, question_num, question_text)\ 
VALUES('NUM1', '1', 'Are cats better than dogs?');"
cur.execute(query)

Now a new ballot has been made and will be available for a folk to vote on if they choose to do so.

# Activity 2

Now that a new ballot has been made, suppose a folk would like to register for the new ballot. To register for a ballot, the folk must supply the ballot name, center acronym, their id, and voting date.

In [ ]:
query = "INSERT INTO registry(ballot_name, center_acronym, folk_id, voting_date)\
VALUES('NUM1', 'UMBC', '0000000000000000', '2022-11-01');"
cur.execute(query)

Now the folk has been register to the newly created ballot and can now cast a vote for the ballot.

# Activity 3

Now that the ballot has been created and a folk has registered for the ballot, the folk can now cast a vote for the ballot 
question. To cast a vote, the folk must provide the ballot name, voting center acronym, their id, the id of the monitor working
at the voting cetner that day, and the date they cast their ballot.

In [ ]:
query = "INSERT INTO casted_vote(ballot_name, center_acronym, folk_id, monitor_id, cast_date)
VALUES('NUM1', 'UMBC', '0000000000000000', '1000000000000000', '2022-11-01');"
cur.execute(query)

# Activity 4

A staff can also update the availability period of a ballot

In [ ]:
query = "UPDATE ballots SET date_start = '2022-12-01 00:00:00', date_end = '2022-12-30 00:00:00' WHERE\
ballot_name = 'NUM1';"
cur.execute(query)

# Activity 5

A staff can also remove a folk. To do this, the staff must first delete any extra information dependant on the folk's profile
existing. This includes things like their phone numbers, email addresses, and other things of that nature.

In [ ]:
query = "DELETE FROM folk_emailAddrs WHERE folk_id = '0000000000000000';"
cur.execute(query)
query = "DELETE FROM folk_phoneNums WHERE folk_id = '0000000000000000';"
cur.execute(query)
query = "DELETE FROM resides_at WHERE folk_id = '0000000000000000';"
cur.execute(query)
query = "DELETE FROM ballot_answers WHERE folk_id = '0000000000000000';"
cur.execute(query)
query = "DELETE FROM casted_vote WHERE folk_id = '0000000000000000';"
cur.execute(query)
query = "DELETE FROM registry WHERE folk_id = '0000000000000000';"
cur.execute(query)
query = "DELETE FROM folk WHERE folk_id = '0000000000000000';"
cur.execute(query)

# Queries

In [ ]:
query = "SELECT first_name, last_name, email, addr_city \
FROM folk INNER JOIN folk_emailaddrs INNER JOIN resides_at\ 
WHERE folk.folk_id = folk_emailaddrs.folk_id AND folk.folk_id = resides_at.folk_id;"
cur.execute(query)

In [ ]:
query = "SELECT addr_city, addr_state, COUNT(*)\
FROM resides_at\
GROUP BY addr_city\
ORDER BY COUNT(*) DESC;"
cur.execute(query)

In [ ]:
query = "SELECT place.addr_state, COUNT(*)\
FROM place INNER JOIN residence\
WHERE place.addr_state = residence.addr_state AND place.addr_city = residence.addr_city AND \
place.addr_street_number = residence.addr_street_number AND place.addr_street_name = residence.addr_street_name\
GROUP BY place.addr_state\
ORDER BY place.addr_state;"
cur.execute(query)

In [ ]:
query = "SELECT folk.folk_id, registry.voting_date\
FROM folk INNER JOIN registry\
WHERE folk.folk_id = registry.folk_id;"
cur.execute(query)

In [ ]:
query = "SELECT voting_center.center_acronym, COUNT(*)\
FROM voting_center INNER JOIN place INNER JOIN place_coords INNER JOIN registry\
WHERE place.addr_state = voting_center.addr_state AND place.addr_city = voting_center.addr_city AND \
place.addr_street_number = voting_center.addr_street_number AND place.addr_street_name = voting_center.addr_street_name\
AND place.coord_id = place_coords.coord_id\
AND (SQRT( POW(place_coords.Y, 2) + POW(place_coords.X, 2) ) <= 5)\
AND registry.center_acronym = voting_center.center_acronym \
AND MONTH(registry.voting_date) = 11\
GROUP BY voting_center.center_acronym\
ORDER BY COUNT(*);"
cur.execute(query)

In [ ]:
query = "SELECT voting_center.center_acronym, place.addr_city, COUNT(*) as c\
FROM voting_center INNER JOIN place INNER JOIN registry\
WHERE voting_center.center_acronym = registry.center_acronym\
AND MONTH(registry.voting_date) = 11 AND (1 <= DAY(registry.voting_date) <= 30)\
AND place.addr_state = voting_center.addr_state AND place.addr_city = voting_center.addr_city AND \
place.addr_street_number = voting_center.addr_street_number AND place.addr_street_name = voting_center.addr_street_name\
GROUP BY place.addr_city\
ORDER BY c DESC;"
cur.execute(query)

In [ ]:
query = "SELECT folk.folk_id, voting_center.addr_state\
FROM folk INNER JOIN registry ON folk.folk_id = registry.folk_id INNER JOIN voting_center ON voting_center.center_acronym = registry.center_acronym\
WHERE registry.center_acronym = voting_center.center_acronym AND folk.folk_id = registry.folk_id AND voting_center.addr_state = 'Maryland'\
GROUP BY folk.folk_id, registry.center_acronym\
HAVING COUNT(*) = COUNT(DISTINCT voting_center.center_acronym);"
cur.execute(query)

In [ ]:
query = "WITH reg_tab AS (SELECT place_coords.X, place_coords.Y, voting_center.center_acronym, folk.folk_id\
FROM voting_center INNER JOIN place_coords INNER JOIN place INNER JOIN registry INNER JOIN folk\
WHERE place.coord_id = place_coords.coord_id\
AND place.addr_state = voting_center.addr_state AND place.addr_city = voting_center.addr_city \
AND place.addr_state = voting_center.addr_state AND place.addr_street_number = voting_center.addr_street_number\
AND registry.center_acronym = voting_center.center_acronym\
AND registry.folk_id = folk.folk_id\
ORDER BY voting_center.center_acronym),\
res_tab AS (SELECT place_coords.X, place_coords.Y, folk.folk_id\
FROM resides_at INNER JOIN place_coords INNER JOIN place INNER JOIN folk\
WHERE place.coord_id = place_coords.coord_id\
AND place.addr_state = resides_at.addr_state AND place.addr_city = resides_at.addr_city \
AND place.addr_state = resides_at.addr_state AND place.addr_street_number = resides_at.addr_street_number\
AND resides_at.folk_id = folk.folk_id\
GROUP BY folk.folk_id)\
SELECT folk.folk_id, res_tab.X, res_tab.Y, reg_tab.X, reg_tab.Y\
FROM folk, res_tab, reg_tab\
WHERE res_tab.folk_id = reg_tab.folk_id\
AND res_tab.folk_id = folk.folk_id\
AND (SQRT( POW(reg_tab.Y-res_tab.Y, 2) + POW(reg_tab.X-res_tab.X, 2) ) > 0)\
GROUP BY folk.folk_id;"
cur.execute(query)

In [ ]:
query = "WITH vc_tab AS (SELECT place_coords.X, place_coords.Y, voting_center.center_acronym\
FROM voting_center INNER JOIN place_coords INNER JOIN place INNER JOIN folk\
WHERE place.coord_id = place_coords.coord_id\
AND place.addr_state = voting_center.addr_state AND place.addr_city = voting_center.addr_city \
AND place.addr_state = voting_center.addr_state AND place.addr_street_number = voting_center.addr_street_number\
ORDER BY voting_center.center_acronym),\
res_tab AS (SELECT place_coords.X, place_coords.Y, folk.folk_id\
FROM resides_at INNER JOIN place_coords INNER JOIN place INNER JOIN folk\
WHERE place.coord_id = place_coords.coord_id\
AND place.addr_state = resides_at.addr_state AND place.addr_city = resides_at.addr_city \
AND place.addr_state = resides_at.addr_state AND place.addr_street_number = resides_at.addr_street_number\
AND resides_at.folk_id = folk.folk_id\
GROUP BY folk.folk_id)\
SELECT vc_tab.center_acronym, folk.folk_id, (SQRT( POW(vc_tab.Y-res_tab.Y, 2) + POW(vc_tab.X-res_tab.X, 2) )) AS distance\
FROM folk INNER JOIN vc_tab INNER JOIN res_tab INNER JOIN operating_period INNER JOIN voting_center\
WHERE DATE(operating_period.date_start) <= '2022-11-01 08:00:00' <= DATE(operating_period.date_end)\
AND folk.folk_id = res_tab.folk_id\
AND voting_center.center_acronym = vc_tab.center_acronym\
GROUP BY voting_center.center_acronym\
ORDER BY distance ASC;"
cur.execute(query)